In [ ]:
!pip uninstall -y transformers
!pip install transformers

import os
os.kill(os.getpid(), 9)  # 🔁 Restarts the Python kernel


In [6]:
import os
os.environ["TRANSFORMERS_NO_TF"] = "1"

#!pip install -q datasets seqeval
import transformers
from transformers import (
    AutoTokenizer,
    AutoModelForTokenClassification,
    TrainingArguments,
    Trainer,
    DataCollatorWithPadding
)
from datasets import Dataset
import numpy as np
from seqeval.metrics import classification_report, f1_score, accuracy_score, precision_score, recall_score
import torch
import pandas as pd
import re

print("Transformers Version:", transformers.__version__)
print("CUDA Available?", torch.cuda.is_available())
# ✅ Load your data
import pandas as pd
df = pd.read_csv("D:/ML/grok_adddress_parser/grok_address_parser-env/bio_tagged_data_108361_141435_eval.csv")  # Make sure labels are space-separated strings
df.rename(columns = {'sentence':'text', 'tags':'labels'}, inplace=True)
df.describe()


Transformers Version: 4.53.2
CUDA Available? False


,text,labels
count,33073,33073
unique,32108,21965
top,"mcd zonal office , shdn zone , near shyamlal c...",B-HOUSE_NUMBER I-HOUSE_NUMBER I-HOUSE_NUMBER I...
freq,15,155


In [ ]:
#to count the top 100 most common tokenns whose frequencies are above 50
from collections import Counter
# For example, assume addresses are in df["address"]
tokens = [tok for addr in df["text"] for tok in addr.split()]
common_tokens = Counter(tokens).most_common(100)
for token, freq in common_tokens:
    if token.isalpha() and freq > 50:
        print(token)
        

no
delhi
gali
f
nagar
vihar
floor
kh
a
g
b
karawal
block
c
d
e
old
laxmi
sonia
s
ground
h
mandawali
pur
first
fazalpur
colony
plot
west
shakarpur
mustafabad
road
shiv
pusta
second
ram
new
nehru
main
third
khasra
vinod
school
extn
near
ganesh
hno
shri
ph
khas
pandav
khajoori
k
khno
park
bhagirathi
side
rajeev
extension
part
j
gandhi
johri
dayalpur
chand
bagh
marg
ngr
dayal
puri


In [3]:
# Create label mappings
label_list = sorted(set(label for row in df["labels"] for label in row.split()))
label2id = {label: i for i, label in enumerate(label_list)}
id2label = {i: label for label, i in label2id.items()}
NUM_LABELS = len(label_list)

print("\n",label_list)
print("\n",id2label)
print("\n",label2id)

print("\n",NUM_LABELS)
# Map string tags to ID list
df["label_ids"] = df["labels"].apply(lambda x: [label2id[l] for l in x.split()])
print(df[["text", "labels", "label_ids"]].head(1))




 ['B-APARTMENT', 'B-AREA', 'B-BLOCK', 'B-FLOOR', 'B-HOUSE_NUMBER', 'B-KHASRA_NUMBER', 'B-LANDMARK', 'B-LOCALITY', 'B-LOCALITY2', 'B-PINCODE', 'B-PLOT_NUMBER', 'B-ROAD_DETAILS', 'B-VILLAGE', 'I-APARTMENT', 'I-AREA', 'I-BLOCK', 'I-FLOOR', 'I-HOUSE_NUMBER', 'I-KHASRA_NUMBER', 'I-LANDMARK', 'I-LOCALITY', 'I-LOCALITY2', 'I-PLOT_NUMBER', 'I-ROAD_DETAILS', 'I-VILLAGE', 'O']

 {0: 'B-APARTMENT', 1: 'B-AREA', 2: 'B-BLOCK', 3: 'B-FLOOR', 4: 'B-HOUSE_NUMBER', 5: 'B-KHASRA_NUMBER', 6: 'B-LANDMARK', 7: 'B-LOCALITY', 8: 'B-LOCALITY2', 9: 'B-PINCODE', 10: 'B-PLOT_NUMBER', 11: 'B-ROAD_DETAILS', 12: 'B-VILLAGE', 13: 'I-APARTMENT', 14: 'I-AREA', 15: 'I-BLOCK', 16: 'I-FLOOR', 17: 'I-HOUSE_NUMBER', 18: 'I-KHASRA_NUMBER', 19: 'I-LANDMARK', 20: 'I-LOCALITY', 21: 'I-LOCALITY2', 22: 'I-PLOT_NUMBER', 23: 'I-ROAD_DETAILS', 24: 'I-VILLAGE', 25: 'O'}

 {'B-APARTMENT': 0, 'B-AREA': 1, 'B-BLOCK': 2, 'B-FLOOR': 3, 'B-HOUSE_NUMBER': 4, 'B-KHASRA_NUMBER': 5, 'B-LANDMARK': 6, 'B-LOCALITY': 7, 'B-LOCALITY2': 8, 'B-PINC

In [ ]:

# 🧠 Step 2: Tokenizer + Model
#model_checkpoint= "/kaggle/input/muril-results/muril_60k_7epoch"
model_checkpoint = "google/muril-base-cased"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
print(len(tokenizer))
model = AutoModelForTokenClassification.from_pretrained(model_checkpoint, num_labels=NUM_LABELS)  # Set this later
data_collator=DataCollatorWithPadding(tokenizer=tokenizer,return_tensors="pt")
# Sample Hinglish address tokens you’ve seen getting split up
new_tokens =[
    "mustafabad", "karawal", "nagar", "sonia", "vihar", "ngr", "fazalpur",
    "mandawali", "marg", "shankar", "khajoori", "khas", "gandhi", "ganga",
    "pur", "laxmi", "madhuban", "aruna", "shakarpur", "johri", "nehru",
    "bhagirathi", "dayalpur","cghs","ganj","patpar","pusta","vinod","fazalpur","pandav"
]
added = tokenizer.add_tokens(new_tokens)
print(f"✅ Added {added} new tokens to the tokenizer.")
model.resize_token_embeddings(len(tokenizer))

print(df[["text", "labels", "label_ids"]].head(1))


tokenizer_config.json:   0%|          | 0.00/206 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/411 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/113 [00:00<?, ?B/s]

197285


pytorch_model.bin:   0%|          | 0.00/953M [00:00<?, ?B/s]

In [7]:
import re
from datasets import Dataset



# 🪄 Step 1: Tokenize text into words
df["tokens"] = df["text"].apply(lambda x: re.findall(r"\w+|[^\w\s]", x))


df["num_tokens"] = df["tokens"].apply(len)
df["num_labels"] = df["label_ids"].apply(len)
mismatched_rows = df[df["num_tokens"] != df["num_labels"]]
print(f"🔍 Rows with token/label mismatch: {len(mismatched_rows)}")
print(mismatched_rows[["text", "tokens", "label_ids"]].head(10))
df.drop(index=[4,16,66479], inplace=True)
#235572,6098,36417,53991,63119,124552 
# 🧠 Step 2: Convert to HuggingFace Dataset
dataset = Dataset.from_pandas(df[["tokens","label_ids"]])
print(df[["text", "tokens"]].head(1),"\n")
print(dataset,"\n")

# 🧠 Step 3: Tokenize and align labels properly
def tokenize_and_align_labels(example):
    tokenized = tokenizer(
        example["tokens"],
        is_split_into_words=True,
        truncation=True,
        padding='max_length',
        max_length=200
    )

    word_ids = tokenized.word_ids()
    label_ids = []
    previous_word_idx = None

    label_all_tokens = True  # 👈 Enable this at the top

    for word_idx in word_ids:
        if word_idx is None:
            label_ids.append(-100)
        else:
            current_label = example["label_ids"][word_idx]
            if not label_all_tokens and word_idx == previous_word_idx:
                label_ids.append(-100)
            else:
                label_ids.append(current_label)
        previous_word_idx = word_idx

    # ✅ Return only what the model expects
    return {
        "input_ids": tokenized["input_ids"],
        "attention_mask": tokenized["attention_mask"],
        "token_type_ids": tokenized.get("token_type_ids", [0] * 128),
        "labels": label_ids
    }

# 🚀 Step 4: Apply tokenizer + cleanup
tokenized_dataset = dataset.map(
    tokenize_and_align_labels,
    batched=False,
    remove_columns=["tokens", "label_ids"]
)
print(tokenized_dataset)


🔍 Rows with token/label mismatch: 3
                                                    text  \
4      old no - 271 , new b - 75 , block b , g no - 5...   
16     old no b - 215 , new b - 197 / 2 , chandu naga...   
66479  uppar bal g - 4 , hn200 , gn4 pusta - 5 sonia ...   

                                                  tokens  \
4      [old, no, -, 271, ,, new, b, -, 75, ,, block, ...   
16     [old, no, b, -, 215, ,, new, b, -, 197, /, 2, ...   
66479  [uppar, bal, g, -, 4, ,, hn200, ,, gn4, pusta,...   

                                               label_ids  
4      [4, 17, 25, 17, 17, 25, 2, 15, 25, 11, 23, 23,...  
16     [4, 17, 17, 25, 17, 17, 17, 17, 25, 7, 20, 25,...  
66479  [25, 25, 4, 17, 17, 25, 17, 25, 11, 7, 20, 25, 9]  
                                                text  \
0  e - 1 / 1091 , shivaji marg , sonia vihar , de...   

                                              tokens  
0  [e, -, 1, /, 1091, ,, shivaji, marg, ,, sonia,...   

Dataset({
    feature

Map:   0%|          | 0/120761 [00:00<?, ? examples/s]

Dataset({
    features: ['__index_level_0__', 'input_ids', 'attention_mask', 'token_type_ids', 'labels'],
    num_rows: 120761
})


In [8]:
import torch
import torch.nn.functional as F

class FocalLoss(torch.nn.Module):
    """
    Focal Loss for multi‑class token classification.
    γ (gamma) amplifies the effect on hard examples.
    α (alpha) is a per‑class weight (use your class_weights tensor here).
    """
    def __init__(self, alpha=None, gamma=2.0, ignore_index=-100, reduction="mean"):
        super().__init__()
        self.alpha = alpha          # tensor of shape [num_labels] or None
        self.gamma = gamma
        self.ignore_index = ignore_index
        self.reduction = reduction

    def forward(self, logits, targets):
        # logits: [N, C]  targets: [N]
        ce_loss = F.cross_entropy(
            logits, targets,
            weight=self.alpha,
            ignore_index=self.ignore_index,
            reduction="none"
        )
        # Only keep valid positions
        valid_mask = (targets != self.ignore_index).float()
        probs = F.softmax(logits, dim=-1)
        targets_one_hot = F.one_hot(targets.clamp(min=0), num_classes=logits.size(-1)).float()
        pt = (probs * targets_one_hot).sum(dim=-1)

        focal_term = (1 - pt) ** self.gamma
        loss = focal_term * ce_loss * valid_mask

        if self.reduction == "mean":
            return loss.sum() / valid_mask.sum()
        else:
            return loss.sum()


In [9]:
# 🧠 Step 5: Training Arguments + Trainer
from transformers import TrainingArguments,logging
import torch

from torch.nn import CrossEntropyLoss
from transformers import Trainer


# 🆕  B‑1: Custom trainer that plugs weighted CE‑loss
class WeightedNERTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False,**kwargs):
        labels  = inputs.get("labels")
        outputs = model(**inputs)
        logits  = outputs.get("logits")
        focal_loss_fct = FocalLoss(alpha=class_weights, gamma=2.0)
        loss = focal_loss_fct(logits.view(-1, logits.size(-1)),
                        labels.view(-1))
        return (loss, outputs) if return_outputs else loss

transformers.logging.set_verbosity_debug()

training_args = TrainingArguments(
    output_dir="./muril_ner_logs",
    run_name="muril_ner_v1",
    per_device_train_batch_size=32,
    per_device_eval_batch_size=64,
    num_train_epochs=6,                 # ✅ Let trainer handle epochs
    logging_dir="./logs",
    logging_strategy="epoch",          # ✅ Log every epoch
    eval_strategy="epoch",       # ✅ Evaluate every epoch
    save_strategy="epoch",             # ✅ Save every epoch
    learning_rate=1e-5,
    lr_scheduler_type="cosine",  # 👈 cosine schedule
    warmup_ratio=0.06,
    weight_decay=0.01,#During each optimiser step the parameters are nudged toward zero,Prevents a few neurons from dominating and over‑fitting rare quirks in the data.
    load_best_model_at_end=True,       # 👈 Optional: restores best eval F1
    metric_for_best_model="f1",        # 👈 requires compute_metrics
    greater_is_better=True,
    report_to="none",
    fp16=True,
    gradient_accumulation_steps=2,
)


def compute_metrics(p):
    predictions, labels = p
    preds = np.argmax(predictions, axis=2)
    
    true_labels = [[id2label[label] for label in example if label != -100] for example in labels]
    true_preds = [[id2label[pred] for pred, label in zip(example_pred, example_label) if label != -100]
                  for example_pred, example_label in zip(preds, labels)]

    return {
        "precision": precision_score(true_labels, true_preds),
        "recall": recall_score(true_labels, true_preds),
        "f1": f1_score(true_labels, true_preds),
        "accuracy": accuracy_score(true_labels, true_preds),
    }
# 🔥 Correct one-time split
split_dataset = tokenized_dataset.train_test_split(test_size=0.2, seed=42)
train_dataset = split_dataset["train"]
eval_dataset = split_dataset["test"]
print("Train size:", len(train_dataset))
print("Eval size:", len(eval_dataset))
print(train_dataset)

#calculating class-weights from train set
from collections import Counter
# 🔧 NEW: compute class weights with clipping on dominant class (like 'O')
from collections import Counter

def compute_clipped_class_weights(train_ds,
                                  label2id,
                                  pad_id=-100,
                                  target_max=1.0,
                                  eps=1e-9):
    """
    Computes inverse-frequency weights, clipped so dominant tag (like 'O') ≤ target_max.
    """
    # Count label frequencies
    counts = Counter()
    for sample in train_ds:
        for lid in sample["labels"]:
            if lid != pad_id:
                counts[lid] += 1

    total = sum(counts.values())

    # Inverse frequency weights
    raw = {i: 1.0 / (counts.get(i, 1) / total + eps) for i in range(len(label2id))}

    # Normalize weights
    max_raw = max(raw.values())
    norm = {i: w / max_raw for i, w in raw.items()}

    # Clip dominant tag weight (usually 'O')
    o_id = label2id["O"]
    if norm[o_id] > target_max:
        norm[o_id] = target_max

    # Convert to tensor
    weight_tensor = torch.tensor(
        [norm.get(i, 1.0) for i in range(len(label2id))]
    ).to("cuda")

    print("✅ Clipped class weights:")
    for i in range(len(label2id)):
        print(f"{id2label[i]:15s}: {weight_tensor[i].item():.4f}")
    
    return weight_tensor

# 💥 Generate clipped weights from your actual training set
class_weights = compute_clipped_class_weights(train_dataset, label2id, target_max=1.0)



from torch.utils.data import DataLoader
# ✅ Basic sanity test: Can we load a few batches without hanging?
loader = DataLoader(train_dataset, batch_size=2,collate_fn=data_collator)
for i, batch in enumerate(loader):
    print(f"✅ Batch {i} loaded")
    if i == 2:
        break


from transformers import EarlyStoppingCallback
# ✅ Pass to trainer
trainer = WeightedNERTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=2)],
    #callbacks=[GradientPrinterCallback()]
)

 

PyTorch: setting up devices


Train size: 96608
Eval size: 24153
Dataset({
    features: ['__index_level_0__', 'input_ids', 'attention_mask', 'token_type_ids', 'labels'],
    num_rows: 96608
})
✅ Clipped class weights:
B-APARTMENT    : 0.1912
B-AREA         : 0.0647
B-BLOCK        : 0.1125
B-FLOOR        : 0.0277
B-HOUSE_NUMBER : 0.0146
B-KHASRA_NUMBER: 0.0221
B-LANDMARK     : 0.2524
B-LOCALITY     : 0.0147
B-LOCALITY2    : 0.1256
B-PINCODE      : 0.0076
B-PLOT_NUMBER  : 0.1228
B-ROAD_DETAILS : 0.0201
B-VILLAGE      : 0.4730
I-APARTMENT    : 0.1211
I-AREA         : 0.0699
I-BLOCK        : 0.0718
I-FLOOR        : 0.0179
I-HOUSE_NUMBER : 0.0047
I-KHASRA_NUMBER: 0.0109
I-LANDMARK     : 0.1544
I-LOCALITY     : 0.0135
I-LOCALITY2    : 0.1512
I-PLOT_NUMBER  : 0.0348
I-ROAD_DETAILS : 0.0055
I-VILLAGE      : 1.0000
O              : 0.0025
✅ Batch 0 loaded
✅ Batch 1 loaded
✅ Batch 2 loaded


/tmp/ipykernel_35/763231292.py:128: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `WeightedNERTrainer.__init__`. Use `processing_class` instead.
  trainer = WeightedNERTrainer(
Using auto half precision backend


In [10]:
from transformers.trainer_utils import set_seed
#3013 0.97it/s  
trainer.train()        
                                                                 

Currently training with a batch size of: 32
The following columns in the training set don't have a corresponding argument in `BertForTokenClassification.forward` and have been ignored: __index_level_0__. If __index_level_0__ are not expected by `BertForTokenClassification.forward`,  you can safely ignore this message.
***** Running training *****
  Num examples = 96,608
  Num Epochs = 6
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 64
  Gradient Accumulation steps = 2
  Total optimization steps = 9,054
  Number of trainable parameters = 237,000,218


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,0.045000,0.038421,0.761304,0.854306,0.805128,0.844295
2,0.031500,0.024381,0.760200,0.854029,0.804388,0.850645
3,0.019700,0.015654,0.769295,0.863440,0.813653,0.854588
4,0.013700,0.012126,0.763615,0.858752,0.808394,0.853434
5,0.011400,0.010996,0.766545,0.860463,0.810793,0.855166


The following columns in the evaluation set don't have a corresponding argument in `BertForTokenClassification.forward` and have been ignored: __index_level_0__. If __index_level_0__ are not expected by `BertForTokenClassification.forward`,  you can safely ignore this message.

***** Running Evaluation *****
  Num examples = 24153
  Batch size = 64
Saving model checkpoint to ./muril_ner_logs/checkpoint-1510
Configuration saved in ./muril_ner_logs/checkpoint-1510/config.json
Model weights saved in ./muril_ner_logs/checkpoint-1510/model.safetensors
tokenizer config file saved in ./muril_ner_logs/checkpoint-1510/tokenizer_config.json
Special tokens file saved in ./muril_ner_logs/checkpoint-1510/special_tokens_map.json
The following columns in the evaluation set don't have a corresponding argument in `BertForTokenClassification.forward` and have been ignored: __index_level_0__. If __index_level_0__ are not expected by `BertForTokenClassification.forward`,  you can safely ignore this messag

TrainOutput(global_step=7550, training_loss=0.024251962118590903, metrics={'train_runtime': 9840.8798, 'train_samples_per_second': 58.902, 'train_steps_per_second': 0.92, 'total_flos': 4.9314129770112e+16, 'train_loss': 0.024251962118590903, 'epoch': 5.0})

In [11]:

# ✅ Save model and tokenizer after training completes
model_save_path = "/kaggle/working/muril_ner_final"
trainer.save_model(model_save_path)
tokenizer.save_pretrained(model_save_path)
import shutil
shutil.make_archive("/kaggle/working/muril_ner_final", 'zip', model_save_path)

  

Saving model checkpoint to /kaggle/working/muril_ner_final
Configuration saved in /kaggle/working/muril_ner_final/config.json
Model weights saved in /kaggle/working/muril_ner_final/model.safetensors
tokenizer config file saved in /kaggle/working/muril_ner_final/tokenizer_config.json
Special tokens file saved in /kaggle/working/muril_ner_final/special_tokens_map.json
tokenizer config file saved in /kaggle/working/muril_ner_final/tokenizer_config.json
Special tokens file saved in /kaggle/working/muril_ner_final/special_tokens_map.json


'/kaggle/working/muril_ner_final.zip'

In [ ]:

from IPython.display import FileLink

# Display a download link in Kaggle
FileLink(/kaggle/working/muril_ner_final.zip)

In [ ]:
from transformers import AutoTokenizer, AutoModelForTokenClassification
import torch
#LOAD EVAL DATASET
df = pd.read_csv("bio_tagged_data_108361_141435_eval.csv")  # Make sure labels are space-separated strings
df.rename(columns = {'sentence':'text', 'tags':'labels'}, inplace=True)
df.describe()

# ✅ Load the saved model + tokenizer 
model_path = "D:\ML\grok_adddress_parser\grok_address_parser-env\muril_120k_5epochs"
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForTokenClassification.from_pretrained(model_path)
from transformers import pipeline
ner_pipe = pipeline("ner", model=model, tokenizer=tokenizer, aggregation_strategy="simple",device=0)  # 'simple' groups tokens into entities

# Create label mappings
label_list = sorted(set(label for row in df["labels"] for label in row.split()))
label2id = {label: i for i, label in enumerate(label_list)}
id2label = {i: label for label, i in label2id.items()}
NUM_LABELS = len(label_list)

print("\n",label_list)
print("\n",id2label)
print("\n",label2id)

print("\n",NUM_LABELS)
# Map string tags to ID list
df["label_ids"] = df["labels"].apply(lambda x: [label2id[l] for l in x.split()])
print(df[["text", "labels", "label_ids"]].head(1))
df.head()

print(id2label)
model.config.id2label = id2label
model.config.label2id = {v: k for k, v in id2label.items()}
#final_eval = trainer.evaluate()
#print("✅ Final Evaluation:", final_eval)

loading file vocab.txt
loading file tokenizer.json
loading file added_tokens.json
loading file special_tokens_map.json
loading file tokenizer_config.json
loading file chat_template.jinja
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
loading configuration file /kaggle/working/muril_ner_final/config.json
Model config BertConfig {
  "architectures": [
    "BertForTokenClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "embedding_size": 768,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6",
    "7": "LABEL_7",
    "8": "LABEL_8",
    "9": "LABEL_9",
    "10": "LABEL_10",
    "11": "LABEL_11",
    "12": "LABEL_12",
    "13": "LABEL_13",
    "14": "LABEL_14",
    "15": "LABEL_15",
    "16": "LABEL_16",


 ['B-APARTMENT', 'B-AREA', 'B-BLOCK', 'B-FLOOR', 'B-HOUSE_NUMBER', 'B-KHASRA_NUMBER', 'B-LANDMARK', 'B-LOCALITY', 'B-LOCALITY2', 'B-PINCODE', 'B-PLOT_NUMBER', 'B-ROAD_DETAILS', 'B-VILLAGE', 'I-APARTMENT', 'I-AREA', 'I-BLOCK', 'I-FLOOR', 'I-HOUSE_NUMBER', 'I-KHASRA_NUMBER', 'I-LANDMARK', 'I-LOCALITY', 'I-LOCALITY2', 'I-PLOT_NUMBER', 'I-ROAD_DETAILS', 'I-VILLAGE', 'O']

 {0: 'B-APARTMENT', 1: 'B-AREA', 2: 'B-BLOCK', 3: 'B-FLOOR', 4: 'B-HOUSE_NUMBER', 5: 'B-KHASRA_NUMBER', 6: 'B-LANDMARK', 7: 'B-LOCALITY', 8: 'B-LOCALITY2', 9: 'B-PINCODE', 10: 'B-PLOT_NUMBER', 11: 'B-ROAD_DETAILS', 12: 'B-VILLAGE', 13: 'I-APARTMENT', 14: 'I-AREA', 15: 'I-BLOCK', 16: 'I-FLOOR', 17: 'I-HOUSE_NUMBER', 18: 'I-KHASRA_NUMBER', 19: 'I-LANDMARK', 20: 'I-LOCALITY', 21: 'I-LOCALITY2', 22: 'I-PLOT_NUMBER', 23: 'I-ROAD_DETAILS', 24: 'I-VILLAGE', 25: 'O'}

 {'B-APARTMENT': 0, 'B-AREA': 1, 'B-BLOCK': 2, 'B-FLOOR': 3, 'B-HOUSE_NUMBER': 4, 'B-KHASRA_NUMBER': 5, 'B-LANDMARK': 6, 'B-LOCALITY': 7, 'B-LOCALITY2': 8, 'B-PINC

In [13]:


print(ner_pipe("House No. 17-B, Plot No. 45, Old Plot No. 22, Third Floor, Gali No. 4, Street No. 2, Mohalla Shyam Nagar, Main Road, KH No. 78/3/2, B Block, Shyam Residency Apartment, Near Shiv Mandir, Shyam Nagar, Laxmi Nagar, Patparganj Village, Delhi - 110092"))

Disabling tokenizer parallelism, we're using DataLoader multithreading already


[{'entity_group': 'HOUSE_NUMBER', 'score': 0.3710264, 'word': 'House No. 17 - B', 'start': 0, 'end': 14}, {'entity_group': 'PLOT_NUMBER', 'score': 0.32253975, 'word': 'Plot No. 45, Old Plot No. 22', 'start': 16, 'end': 44}, {'entity_group': 'FLOOR', 'score': 0.3685306, 'word': 'Third Floor', 'start': 46, 'end': 57}, {'entity_group': 'ROAD_DETAILS', 'score': 0.35970134, 'word': 'Ga', 'start': 59, 'end': 61}, {'entity_group': 'ROAD_DETAILS', 'score': 0.3228927, 'word': '##li No. 4, Street No. 2, Mohalla Shyam', 'start': 61, 'end': 98}, {'entity_group': 'LOCALITY', 'score': 0.17051268, 'word': 'Nagar', 'start': 99, 'end': 104}, {'entity_group': 'ROAD_DETAILS', 'score': 0.32380474, 'word': ', Main Road', 'start': 104, 'end': 115}, {'entity_group': 'KHASRA_NUMBER', 'score': 0.36270142, 'word': 'K', 'start': 117, 'end': 118}, {'entity_group': 'KHASRA_NUMBER', 'score': 0.36971754, 'word': '##H No. 78 / 3 / 2', 'start': 118, 'end': 130}, {'entity_group': 'BLOCK', 'score': 0.33427504, 'word': '

In [ ]:
#only evaluation 
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import re
from datasets import Dataset

# 🪄 Step 1: Tokenize text into words
df["tokens"] = df["text"].apply(lambda x: re.findall(r"\w+|[^\w\s]", x))


df["num_tokens"] = df["tokens"].apply(len)
df["num_labels"] = df["label_ids"].apply(len)
mismatched_rows = df[df["num_tokens"] != df["num_labels"]]
print(f"🔍 Rows with token/label mismatch: {len(mismatched_rows)}")
print(mismatched_rows[["text", "tokens", "label_ids"]].head())
#df.drop(index=[4, 16], inplace=True)

# 🧠 Step 2: Convert to HuggingFace Dataset
dataset = Dataset.from_pandas(df[["tokens", "label_ids"]])
print(df[["text", "tokens"]].head(1))

# 🧠 Step 3: Tokenize and align labels properly
def tokenize_and_align_labels(example):
    tokenized = tokenizer(
        example["tokens"],
        is_split_into_words=True,
        truncation=True,
        padding='max_length',
        max_length=200
    )

    word_ids = tokenized.word_ids()
    label_ids = []
    previous_word_idx = None

    label_all_tokens = True  # 👈 Enable this at the top

    for word_idx in word_ids:
        if word_idx is None:
            label_ids.append(-100)
        else:
            current_label = example["label_ids"][word_idx]
            if not label_all_tokens and word_idx == previous_word_idx:
                label_ids.append(-100)
            else:
                label_ids.append(current_label)
        previous_word_idx = word_idx

    # ✅ Return only what the model expects
    return {
        "input_ids": tokenized["input_ids"],
        "attention_mask": tokenized["attention_mask"],
        "token_type_ids": tokenized.get("token_type_ids", [0] * 128),
        "labels": label_ids
    }

# 🚀 Step 4: Apply tokenizer + cleanup
tokenized_dataset = dataset.map(
    tokenize_and_align_labels,
    batched=False,
    remove_columns=["tokens", "label_ids"]
)


#seperate trainer for evaluation 
from transformers import Trainer, TrainingArguments
from sklearn.metrics import classification_report
import numpy as np

args = TrainingArguments(
    output_dir="./results", per_device_eval_batch_size=16
)

def compute_metrics(p):
    preds = np.argmax(p.predictions, axis=2)
    true = p.label_ids
    pred_tags, true_tags = [], []

    for pred_seq, true_seq in zip(preds, true):
        for p, t in zip(pred_seq, true_seq):
            if t != -100:
                pred_tags.append(id2label[p])
                true_tags.append(id2label[t])

    report = classification_report(true_tags, pred_tags, zero_division=0, output_dict=False)
    print(report)
        # 📉 Confusion matrix
    labels = list(id2label.values())  # all label strings in order
    cm = confusion_matrix(true_tags, pred_tags, labels=labels)

    plt.figure(figsize=(14, 12))
    sns.heatmap(cm,
                xticklabels=labels,
                yticklabels=labels,
                cmap="Blues",
                annot=False,
                fmt="d",
                square=True,
                cbar_kws={'label': 'Count'})
    plt.xlabel("Predicted Label")
    plt.ylabel("True Label")
    plt.title("NER Confusion Matrix")
    plt.tight_layout()

    # 💾 Save plot
    os.makedirs("results", exist_ok=True)
    plt.savefig("results/confusion_matrix.png")
    plt.close()
    return {}

trainer = Trainer(model=model, args=args,compute_metrics=compute_metrics)
trainer.evaluate(tokenized_dataset)

🔍 Rows with token/label mismatch: 0
Empty DataFrame
Columns: [text, tokens, label_ids]
Index: []
                                                text  \
0  mohd shafi qureshi755 gno - 10 rajeev gandhi n...   

                                              tokens  
0  [mohd, shafi, qureshi755, gno, -, 10, rajeev, ...  


Map:   0%|          | 0/33073 [00:00<?, ? examples/s]

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).

***** Running Evaluation *****
  Num examples = 33073
  Batch size = 16


                 precision    recall  f1-score   support

    B-APARTMENT       0.45      0.86      0.59      5188
         B-AREA       0.44      0.67      0.53     12204
        B-BLOCK       0.38      0.83      0.52      3659
        B-FLOOR       0.89      0.94      0.91      8459
 B-HOUSE_NUMBER       0.93      0.94      0.93     35562
B-KHASRA_NUMBER       0.75      0.97      0.85     27425
     B-LANDMARK       0.36      0.84      0.50      2101
     B-LOCALITY       0.69      0.73      0.71     31758
    B-LOCALITY2       0.23      0.34      0.28      4435
      B-PINCODE       1.00      1.00      1.00     65958
  B-PLOT_NUMBER       0.39      0.68      0.49      8848
 B-ROAD_DETAILS       0.77      0.68      0.72     27923
      B-VILLAGE       0.16      0.36      0.22      1236
    I-APARTMENT       0.37      0.79      0.50      7392
         I-AREA       0.45      0.74      0.56     13469
        I-BLOCK       0.35      0.88      0.50      5120
        I-FLOOR       0.88    

Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

In [ ]:

df = pd.read_csv("/kaggle/input/address-ner/bio_tagged_data_108361_141435_eval.csv")
from tqdm import tqdm

raw_addresses = df['sentence'].tolist()
#print(raw_addresses)
print(label_list)

# Predict in batches (not all at once)
batch_results = []
for chunk in tqdm(range(0, len(raw_addresses), 16)):
    batch = raw_addresses[chunk:chunk+16]
    batch_results.extend(ner_pipe(batch))
print('batch results',batch_results)
# Then postprocess as before


In [ ]:
import pandas as pd
from collections import defaultdict

# ✏️ 1) Edit this list if you add / drop labels later
FIELDS = [
    "HOUSE_NUMBER", "PLOT_NUMBER", "FLOOR", "ROAD_DETAILS", "KHASRA_NUMBER",
    "BLOCK", "APARTMENT", "LANDMARK", "LOCALITY", "AREA", "LOCALITY2",
    "VILLAGE", "PINCODE"
]

# 🔧 2) Utility to stitch sub‑words: "##no" → "no" (attach to previous token)
def join_word(prev, cur):
    if cur.startswith("##"):
        return prev + cur[2:]        # no space, drop ##
    if prev.endswith("-") or prev.endswith("/"):
        return prev + cur            # e.g. "23/" + "A"
    if prev:                         # normal case: add a space
        return prev + " " + cur
    return cur                       # first token

# 🏗️ 3) Convert one NER output list → dict of fields
def convert_single_output(entity_list):
    row = defaultdict(str)

    for ent in entity_list:
        key  = ent["entity_group"]
        word = ent["word"]
        if row[key]:
            row[key] = join_word(row[key], word)
        else:
            row[key] = word.lstrip("##")  # first token

    # 🌐 Compose complete address (simple comma join)
    ordered_parts = [row[f] for f in FIELDS if row[f]]
    if row["PINCODE"]:
        ordered_parts.append(f"Delhi -{row['PINCODE']}")   
    row["complete_address"] = ", ".join(ordered_parts)

    return row

# 🚀 4) Run on your whole batch_results list
parsed_rows = [convert_single_output(addr_entities) for addr_entities in batch_results]
df = pd.DataFrame(parsed_rows)

# 🔖 5) Re‑order columns and save
df = df[FIELDS + ["complete_address"]]
df.to_csv("parsed_address.csv", index=False)

print("✅ Saved parsed_address.csv")
display(df.head())


In [ ]:
import torch
from transformers import pipeline



# Predict NER on each address
predicted = []
for address in raw_addresses:
    ents = ner_pipe(address)
    predicted.append(ents)

# Let's look!
for i, address in enumerate(raw_addresses):
    print(f"\n📬 {address}")
    for ent in predicted[i]:
        print(f"{ent['word']:<20} ➜ {ent['entity_group']}")


In [ ]:
import pandas as pd
from collections import defaultdict

structured_data = []

for ents in predicted:
    entity_map = defaultdict(str)
    for ent in ents:
        label = ent["entity_group"].lower().replace("b-", "").replace("i-", "")
        entity_map[label] += ent["word"] + " "
    structured_data.append({k.strip(): v.strip() for k, v in entity_map.items()})

df_structured = pd.DataFrame(structured_data)
print(df_structured.describe(),"\n")
print(df_structured.head(10))
